# Weapons Detection Traning on YoloV8

Dataset: https://app.roboflow.com/mike-ovbzu/weapons_detection-tsdac/9

In [ ]:
#!pip install ultralytics==8.0.20
#!pip install onnxruntime

## Load libraries

In [ ]:
import os
import glob
import torch
import shutil
import warnings
import ultralytics
from ultralytics import YOLO
from datetime import datetime
from IPython.display import display, Image


## Turn off Warnings

In [ ]:
warnings.filterwarnings('ignore')

## Check GPU availability

In [ ]:
gpu_avail = torch.cuda.is_available()
print(gpu_avail)

## Display GPU specs

In [ ]:
if gpu_avail:
    !nvidia-smi

In [ ]:
HOME = os.getcwd()
print(HOME)

## Check Ultralytics

In [ ]:
ultralytics.checks()

## Train with Hyperparameter Tuning

### This training gives the best results

In [ ]:
torch.cuda.empty_cache()

## Train the model

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={HOME}"/datasets/data.yaml" epochs=100 imgsz=400 plots=True patience=50 batch=32 lr0=0.001 optimizer=Adam dropout=0.0 device=0 

## Clear CUDA cache

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!ls {HOME}/runs/detect/train/

## Load the image file for a test prediction

In [ ]:
Image(filename=f'{HOME}/runs/detect/train/results.png', width=800, height=500)

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={HOME}/datasets/data.yaml

## Adjust confidence for different results

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.50 source={HOME}/datasets/valid/images save=True

## Display Confusion Matrix

In [ ]:
Image(filename=f'{HOME}/runs/detect/val/confusion_matrix.png', width=800, height=500)

## Run test prediction across 10 images

In [ ]:
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:5]:
      display(Image(filename=image_path, width=600))
      print("\n")

## Load the best performing model

In [ ]:
model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

## Default output format is PyTorch

## Export the model to onnx format

In [ ]:
# Export the model
model.export(format='onnx')
source_file='./runs/detect/train/weights/best.onnx'
# Copy the file with the new name
shutil.move(source_file, './best.onnx')

## Update the trained model name and move it to the root dir

In [ ]:
# Source file path
source_file = './runs/detect/train/weights/best.pt'
last_file = './runs/detect/train/weights/last.pt'
# Destination directory path
destination_directory = './'

# Get the current date and time
now = datetime.now()
date_time = now.strftime("%m_%d_%y_%H_%M")

# Extract the file's extension (e.g., .txt)
file_name, file_extension = os.path.splitext(source_file)

# Construct the new file name with the date-time format
new_file_name = f"weapons_detection_{date_time}{file_extension}"

# Combine the destination directory and the new file name
destination_file = os.path.join(destination_directory, new_file_name)

# Copy the file with the new name
shutil.copy(source_file, destination_file)
os.remove(source_file)
shutil.rmtree('./runs/detect/')